In [1]:
!pip install colabcode
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install python-multipart
!pip install aiofiles

     |████████████████████████████████| 8.3MB 7.4MB/s 
     |████████████████████████████████| 747kB 55.6MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 245kB 54.1MB/s 
     |████████████████████████████████| 430kB 55.6MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
     |████████████████████████████████| 122kB 62.0MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.4-cp37-none-any.whl size=18971 sha256=871072e2981a8a1ca46683294dd7974a01a97feae88715818280f091145fd596
  Stored in directory: /root/.cache/pip/wheels/8a/82/b1/cecfba4ff6e2f05777a5a4a65b46c1114842453d5a0e61bdd4
Successfully built pyngrok
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 6.1 which is incompatible.
  Found existing installatio

     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 10.1MB 9.9MB/s 
  Created wheel for python-multipart: filename=python_multipart-0.0.5-cp37-none-any.whl size=31671 sha256=5cc9dc38edc115936fb84538007cce32bac531374695fdb540e3f6f3042651ff
  Stored in directory: /root/.cache/pip/wheels/f0/e6/66/14a866a3cbd6a0cabfbef91f7edf40aa03595ef6c88d6d1be4
Successfully built python-multipart


In [12]:
!git clone https://github.com/NathanUA/U-2-Net.git
!pip install gdown
!gdown https://drive.google.com/u/1/uc?id=1IG3HdpcRiDoWNookbncQjeaPN28t90yW
!mkdir ./saved_models
!mkdir ./saved_models/u2net_portrait
!mkdir ./test_data
!mkdir ./test_data/test_portrait_images
!mkdir ./test_data/test_portrait_images/your_portrait_im
!mkdir ./test_data/test_portrait_images/portrait_im
!cp u2net_portrait.pth ./saved_models/u2net_portrait

fatal: destination path 'U-2-Net' already exists and is not an empty directory.
Permission denied: https://drive.google.com/u/1/uc?id=1IG3HdpcRiDoWNookbncQjeaPN28t90yW
Maybe you need to change permission over 'Anyone with the link'?
mkdir: cannot create directory ‘./saved_models’: File exists
mkdir: cannot create directory ‘./saved_models/u2net_portrait’: File exists
mkdir: cannot create directory ‘./test_data’: File exists
mkdir: cannot create directory ‘./test_data/test_portrait_images’: File exists
mkdir: cannot create directory ‘./test_data/test_portrait_images/your_portrait_im’: File exists
mkdir: cannot create directory ‘./test_data/test_portrait_images/portrait_im’: File exists
cp: cannot stat 'u2net_portrait.pth': No such file or directory


In [6]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)
from fastapi import FastAPI, File, UploadFile, Request
import pickle
import shutil
from io import BytesIO
from typing import List
from fastapi.responses import StreamingResponse
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.responses import FileResponse
from pydantic import BaseModel
from PIL import Image
import os
import aiofiles
import base64
import json
import base64
app = FastAPI()

def decodeImg(encodedImg):
    pil_image=Image.open(BytesIO(encodedImg))
    return pil_image
@app.post("/files/")
async def image(file:bytes=File(...)):
    filename="./test_data/test_portrait_images/portrait_im/destination.jpeg"
    image=decodeImg(file)

    !rm -rf ./test_data/test_portrait_images/portrait_im
    !rm -rf ./test_data/test_portrait_images/portrait_results
    !mkdir ./test_data/test_portrait_images/portrait_im
    
    image=image.save(filename)

    !python /content/U-2-Net/u2net_portrait_test.py

    arr=os.listdir('./test_data/test_portrait_images/portrait_results')
    # print(arr)
    with open("./test_data/test_portrait_images/portrait_results/destination.png", "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
        print(encoded_string)
        return {"image":encoded_string}
    # return {FileResponse("./test_data/test_portrait_images/portrait_results/destination.png",media_type="image/jpeg")}
# async def image(response:)

def base64str_to_PILImage(base64str):
    base64_img_bytes = base64str.encode('utf-8')
    base64bytes = base64.b64decode(base64_img_bytes)
    bytesObj = BytesIO(base64bytes)
    img = Image.open(bytesObj)
    return img
@app.get("/b64image/")
async def immma(b64enc:str ):
    # retClass=[{"b64image":"encoded str"}]
    print("insideb64")
    print(b64enc)
    # return 0
    filename="./test_data/test_portrait_images/portrait_im/destination.jpeg"
    image=base64str_to_PILImage(b64enc)

    !rm -rf ./test_data/test_portrait_images/portrait_im
    !rm -rf ./test_data/test_portrait_images/portrait_results
    !mkdir ./test_data/test_portrait_images/portrait_im
    
    image=image.save(filename)

    !python /content/U-2-Net/u2net_portrait_test.py

    arr=os.listdir('./test_data/test_portrait_images/portrait_results')
    # print(arr)
    with open("./test_data/test_portrait_images/portrait_results/destination.png", "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
        print(encoded_string)
        return {"image":encoded_string}



@app.get("/")
async def main():
    content = """
<body>
<form action="/files/" enctype="multipart/form-data" method="post">
<input name="files" type="image" >
<input type="submit">
</form>
<form action="/uploadfiles/" enctype="multipart/form-data" method="post">
<input name="files" type="file" multiple>
<input type="submit">
</form>
</body>
    """
    return HTMLResponse(content=content)



In [7]:
server.run_app(app) 

Public URL: NgrokTunnel: "http://3b63723901d1.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [135]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     2409:4043:2c02:eff4:3719:ee0e:e1de:a9e9:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4043:2c02:eff4:3719:ee0e:e1de:a9e9:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2409:4043:2c02:eff4:3719:ee0e:e1de:a9e9:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4043:2c02:eff4:96a1:67e2:4ddb:f56a:0 - "GET /b64image?b64enc=%2F9j%2F4AAQSkZJRgABAQAAAQABAAD%2F2wCEAAYGBgYHBgcICAcKCwoLCg8ODAwODxYQERAREBYiFRkVFRkVIh4kHhweJB42KiYmKjY%2BNDI0PkxERExfWl98fKcBBgYGBgcGBwgIBwoLCgsKDw4MDA4PFhAREBEQFiIVGRUVGRUiHiQeHB4kHjYqJiYqNj40MjQ%2BTERETF9aX3x8p%2F%2FCABEIAeYEOAMBIgACEQEDEQH%2FxAAwAAEAAwEBAQAAAAAAAAAAAAAAAQIDBAUGAQEBAQEBAAAAAAAAAAAAAAAAAQIDBP%2FaAAwDAQACEAMQAAAC%2BfGNAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE2KLii8mbQZtBm0GbWTFuMHRJzOkczpk5XUOV1DldQ5XWOR1ycbtk4XfY856UnmPTqec68zBtEZNRk1gzaDNoM2gzaKzXFFxReIqsqqRCYAAAC9oyajJtBk2GLYYthi2GLYYthi2GLUZNoMmoyajJqMmgzXVRaCEwCCUCVRZUWVFlBdSxIAAAANNua8bTjMu3Vy%2FUWebh6%2Blnj83t6HiX9mp8%2Ff6Sy%2FPbfRXs%2Be096K87H2KJ5k%2BpJ5k%2

insideb64
...
Number of images:  1
...load U2NET---173.6 MB
Traceback (most recent call last):
  File "/content/U-2-Net/u2net_portrait_test.py", line 117, in <module>
    main()
  File "/content/U-2-Net/u2net_portrait_test.py", line 87, in main
    net.load_state_dict(torch.load(model_dir))
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 579, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: './saved_models/u2net_portrait/u2net_portrait.pth'
INFO:     2409:4043:2c02:eff4:96a1:67e2:4ddb:f56a:0 - "GET /b64image/?b64enc=%2F9j%2F4AAQSkZJRgABAQAAAQABAAD%2F2wCEAAYGBgYHBgcICAcKCwoLCg8ODAwODxYQERAREBYiFRkVFRkVIh4k

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/protocols/http/h11_impl.py", line 394, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/applications.py", line 111, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 159, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.7/dist-pa